# Equity Investing -  Fundamental Analysis

1. Scraping key statistical data of Nifty 500 stocks

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Tickers" data-toc-modified-id="Import-Tickers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Tickers</a></span></li><li><span><a href="#Get-Key-Statistics" data-toc-modified-id="Get-Key-Statistics-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get Key Statistics</a></span></li></ul></div>

In [1]:
import time
import urllib
from urllib import request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import requests

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Import Tickers

[Source](https://www.nseindia.com/products/content/equities/indices/about_indices.htm)

Nifty 500 consists of the following:

* Nifty 50
* Nifty Next 50 
* Nifty Midcap 150
* Nift Smallcap 250

In [2]:
def get_ticker_list(url):
    s=requests.get(url).content
    df_ticker=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    cols = ['Company Name', 'Industry', 'Symbol']
    df_ticker = df_ticker[cols]
    return(df_ticker)

In [3]:
url = 'https://www.nseindia.com/content/indices/ind_nifty50list.csv'

nifty_50 = get_ticker_list(url)
nifty_50['category'] = 'Nifty 50'

In [4]:
url = 'https://www.nseindia.com/content/indices/ind_niftynext50list.csv'

nifty_next_50 = get_ticker_list(url)
nifty_next_50['category'] = 'Nifty Next 50'

In [5]:
url = 'https://www.nseindia.com/content/indices/ind_niftymidcap150list.csv'

nifty_midcap_150 = get_ticker_list(url)
nifty_midcap_150['category'] = 'Nifty Midcap 150'

In [6]:
url = 'https://www.nseindia.com/content/indices/ind_niftysmallcap250list.csv'

nifty_smallcap_250 = get_ticker_list(url)
nifty_smallcap_250['category'] = 'Nifty Smallcap 250'

**Merging**

In [7]:
df_tickers = pd.DataFrame()
df_tickers = df_tickers.append(nifty_50)
df_tickers = df_tickers.append(nifty_next_50)
df_tickers = df_tickers.append(nifty_midcap_150)
df_tickers = df_tickers.append(nifty_smallcap_250)
df_tickers.columns = ['company', 'industry', 'symbol', 'category']

display(df_tickers.shape[0])
display(df_tickers.sample(1))

500

,company,industry,symbol,category
58,Eveready Industries India Ltd.,CONSUMER GOODS,EVEREADY,Nifty Smallcap 250


## Get Key Statistics

In [8]:
def get_key_stats(stock):
    with urllib.request.urlopen('https://www.screener.in/company/'+stock) as response:
        html = response.read().decode('utf-8').replace(' ', '')      

    market_cap = html.split('MarketCap:\n\n<iclass="fafa-rupee"></i>\n\n<b>')[1].split('</b>')[0]
    
    current_price = html.split('CurrentPrice:\n\n<iclass="fafa-rupee"></i>\n\n<b>')[1].split('</b>')[0]
    
    high_52week = html.split('52weeksHigh/Low\n<iclass="fafa-rupee"></i><b>')[1].split('</b>/<b>')[0]
    
    low_52week = html.split('52weeksHigh/Low\n<iclass="fafa-rupee"></i><b>')[1].split('</b>/<b>')[1].split('</b>')[0]
    
    book_value = html.split('BookValue:\n\n<iclass="fafa-rupee"></i>\n\n<b>')[1].split('</b>')[0]
    
    per = html.split('StockP/E:\n\n<b>')[1].split('</b>')[0]
    
    dividend_yield = html.split('DividendYield:\n\n<b>')[1].split('</b>')[0]
    
    roce = html.split('ROCE:\n\n<b>')[1].split('</b>')[0]
    
    roe = html.split('ROE:\n\n<b>')[1].split('</b>')[0]
    
    sales_growth_3yr = html.split('SalesGrowth(3Yrs):\n\n<b>')[1].split('</b>')[0]
    
    print('Done:', stock)
    return(list([stock, market_cap, current_price,
                 high_52week, low_52week,
                 book_value, per, dividend_yield, roce, roe, sales_growth_3yr]))
    

In [9]:
def df_stats(df_stocks):
    cols = ['ticker', 'market_cap', 'current_value', 
            'high_52week', 'low_52week',
            'book_value', 'price_earnings', 
            'dividend_yield', 'roce', 'roe', 'sales_growth_3yr']
    
    df = pd.DataFrame()
    i = 0
    
    for ticker in df_stocks:
        data = pd.DataFrame(get_key_stats(ticker)).T
        time.sleep(1)
        
        df = df.append(data)
        i += 1

    df.columns = cols
    df = df.reset_index(drop=True)
    
    return(df)

In [10]:
%%time 
nifty_500_stats = df_stats(df_tickers.symbol)
nifty_500_stats.to_csv('data/nifty_500_stats.csv', 
                       sep=';', encoding='utf-8')

Done: ADANIPORTS
Done: ASIANPAINT
Done: AXISBANK
Done: BAJAJ-AUTO
Done: BAJFINANCE
Done: BAJAJFINSV
Done: BPCL
Done: BHARTIARTL
Done: INFRATEL
Done: CIPLA
Done: COALINDIA
Done: DRREDDY
Done: EICHERMOT
Done: GAIL
Done: GRASIM
Done: HCLTECH
Done: HDFCBANK
Done: HEROMOTOCO
Done: HINDALCO
Done: HINDPETRO
Done: HINDUNILVR
Done: HDFC
Done: ITC
Done: ICICIBANK
Done: IBULHSGFIN
Done: IOC
Done: INDUSINDBK
Done: INFY
Done: JSWSTEEL
Done: KOTAKBANK
Done: LT
Done: M&M
Done: MARUTI
Done: NTPC
Done: ONGC
Done: POWERGRID
Done: RELIANCE
Done: SBIN
Done: SUNPHARMA
Done: TCS
Done: TATAMOTORS
Done: TATASTEEL
Done: TECHM
Done: TITAN
Done: UPL
Done: ULTRACEMCO
Done: VEDL
Done: WIPRO
Done: YESBANK
Done: ZEEL
Done: ABB
Done: ACC
Done: ABCAPITAL
Done: AMBUJACEM
Done: ASHOKLEY
Done: AUROPHARMA
Done: DMART
Done: BANDHANBNK
Done: BANKBARODA
Done: BEL
Done: BHEL
Done: BIOCON
Done: BOSCHLTD
Done: BRITANNIA
Done: CADILAHC
Done: COLPAL
Done: CONCOR
Done: DLF
Done: DABUR
Done: GICRE
Done: GODREJCP
Done: HDFCLIFE
Done

In [11]:
nifty_500_stats = pd.read_csv('data/nifty_500_stats.csv', 
                              sep=';', 
                              encoding='utf-8', 
                              thousands=',',
                              na_values='None'
                             )

del nifty_500_stats['Unnamed: 0']
nifty_500_stats = nifty_500_stats.apply(pd.to_numeric, errors='ignore')
nifty_500_stats = nifty_500_stats.rename(columns = {'ticker':'symbol'})

In [12]:
df_nifty_500 = pd.merge(df_tickers, nifty_500_stats, on='symbol')

display(df_nifty_500.shape)
df_nifty_500.sample(3)

(500, 14)

,company,industry,symbol,category,market_cap,current_value,high_52week,low_52week,book_value,price_earnings,dividend_yield,roce,roe,sales_growth_3yr
169,IIFL Holdings Ltd.,FINANCIAL SERVICES,IIFL,Nifty Midcap 150,14457.0,453.10,872.7,341.0,46.47,78.05,1.10,13.60,13.83,13.01
183,L&T Technology Services Ltd.,IT,LTTS,Nifty Midcap 150,17396.0,1673.00,1852.5,815.1,187.81,28.88,0.94,38.98,28.00,11.05
440,Reliance Naval and Engineering Ltd.,INDUSTRIAL MANUFACTURING,RNAVAL,Nifty Smallcap 250,1044.0,14.15,72.6,11.0,-6.61,NaN,0.00,-3.08,-114.00,-26.12


In [13]:
df_nifty_500.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 14 columns):
company             500 non-null object
industry            500 non-null object
symbol              500 non-null object
category            500 non-null object
market_cap          500 non-null float64
current_value       500 non-null float64
high_52week         497 non-null float64
low_52week          497 non-null float64
book_value          500 non-null float64
price_earnings      455 non-null float64
dividend_yield      500 non-null float64
roce                500 non-null float64
roe                 498 non-null float64
sales_growth_3yr    491 non-null float64
dtypes: float64(10), object(4)
memory usage: 58.6+ KB


In [14]:
df_nifty_500.to_csv('data/nifty_500_stats.csv', 
                       sep=';', encoding='utf-8')